In [1]:
import pandas as pd
import numpy as np
import csv
from datetime import datetime, timedelta

import tensorflow as tf
from transformers import BertTokenizer, BertModel, TFBertForSequenceClassification

In [2]:
company = '台積電'
company2 = '台積'
name = '張忠謀'

In [3]:
df2018=pd.read_csv("bda2019_dataset/TWSE2018.csv", encoding="utf-8")
df2017=pd.read_csv("bda2019_dataset/TWSE2017.csv", encoding="utf-8")
df2016=pd.read_csv("bda2019_dataset/TWSE2016.csv", encoding="utf-8")
df_2330 = pd.concat([df2018,df2017,df2016],axis=0, ignore_index=True)
df_2330=df_2330[df_2330['證券代碼'].str.contains(company)]
df_2330['年月日']=pd.to_datetime(df_2330['年月日'])
df_2330['收盤價(元)']=df_2330['收盤價(元)'].astype(str).str.replace(',','').astype('float64')
df_2330.head(3)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE
269,2330 台積電,2018-12-28,223.5,225.5,222.5,225.5,"30,156","6,762,447","10,247","25,930,380",16.68,3.71
1247,2330 台積電,2018-12-27,222,224,220.5,223.0,"21,915","4,874,845","9,531","25,930,380",16.49,3.67
2225,2330 台積電,2018-12-26,218.5,219.5,216,216.5,"16,893","3,678,108","6,998","25,930,380",16.01,3.56


In [4]:
news=pd.read_csv("bda2019_dataset/news2.csv",encoding="utf-8")
news['content'].fillna(value='哈哈', inplace=True)
news['title'].fillna(value='哈哈', inplace=True)
tnews=news[news['title'].str.contains('張忠謀|台積|台積電')]
tnews=tnews.reset_index(drop=True)
tnews['post_time']=pd.to_datetime(tnews['post_time'])
len(tnews)

3430

In [5]:
df_2330['highpriceshift'] = df_2330['收盤價(元)'].shift(periods=1)
df_2330['div'] = df_2330['highpriceshift'] - df_2330['收盤價(元)']  #只算上漲還是下跌
# df_2330['div'] = (df_2330['highpriceshift'] - df_2330['收盤價(元)'])/df_2330['收盤價(元)']  #算出上漲或下跌率
df_2330=df_2330[1:]
df_2330=df_2330.reset_index(drop=True)
df_2330.head()

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,highpriceshift,div
0,2330 台積電,2018-12-27,222,224,220.5,223.0,"21,915","4,874,845","9,531","25,930,380",16.49,3.67,225.5,2.5
1,2330 台積電,2018-12-26,218.5,219.5,216,216.5,"16,893","3,678,108","6,998","25,930,380",16.01,3.56,223.0,6.5
2,2330 台積電,2018-12-25,215.5,218,215,217.5,"19,402","4,198,767","8,065","25,930,380",16.09,3.58,216.5,-1.0
3,2330 台積電,2018-12-24,221.5,221.5,219.5,220.0,"23,376","5,151,549","6,122","25,930,380",16.27,3.62,217.5,-2.5
4,2330 台積電,2018-12-22,220,221.5,220,221.5,"5,185","1,145,814","2,229","25,930,380",16.38,3.65,220.0,-1.5


In [6]:
df_2330['label']='平'
df_2330.loc[df_2330['div'] > 0.0, 'label'] = '漲'
df_2330.loc[df_2330['div'] < 0.0, 'label'] = '跌'
df_2330.head()

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,highpriceshift,div,label
0,2330 台積電,2018-12-27,222,224,220.5,223.0,"21,915","4,874,845","9,531","25,930,380",16.49,3.67,225.5,2.5,漲
1,2330 台積電,2018-12-26,218.5,219.5,216,216.5,"16,893","3,678,108","6,998","25,930,380",16.01,3.56,223.0,6.5,漲
2,2330 台積電,2018-12-25,215.5,218,215,217.5,"19,402","4,198,767","8,065","25,930,380",16.09,3.58,216.5,-1.0,跌
3,2330 台積電,2018-12-24,221.5,221.5,219.5,220.0,"23,376","5,151,549","6,122","25,930,380",16.27,3.62,217.5,-2.5,跌
4,2330 台積電,2018-12-22,220,221.5,220,221.5,"5,185","1,145,814","2,229","25,930,380",16.38,3.65,220.0,-1.5,跌


In [7]:
for day in range(len(tnews)):
    targetdate=tnews.loc[day,'post_time'].date()
    for day1 in range(len(df_2330)):
        try:
            if targetdate==df_2330.loc[day1,'年月日'].date():
                tnews.loc[day,'label']=df_2330.loc[day1,'label']
            elif (targetdate+timedelta(1))==df_2330.loc[day1,'年月日'].date():
                tnews.loc[day,'label']=df_2330.loc[day1+1,'label']
            elif (targetdate+timedelta(2))==df_2330.loc[day1,'年月日'].date():
                tnews.loc[day,'label']=df_2330.loc[day1+1,'label']
            elif (targetdate+timedelta(3))==df_2330.loc[day1,'年月日'].date():
                tnews.loc[day,'label']=df_2330.loc[day1+1,'label']
            elif (targetdate+timedelta(4))==df_2330.loc[day1,'年月日'].date():
                tnews.loc[day,'label']=df_2330.loc[day1+1,'label']
            elif (targetdate+timedelta(5))==df_2330.loc[day1,'年月日'].date():
                tnews.loc[day,'label']=df_2330.loc[day1+1,'label']
        except:
            continue
tnews.head(3)

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url,label
0,1451621511658_N01,news,yahoo股市,重大要聞,2016-01-01 11:46:00,台積電ADR31日下跌0.13美元跌幅0.57%折台股150.45元,中央社,(中央社台北2016年 1月 1日電)台積電在NYSE掛牌 ADR以 22.75美元作收，下...,https://tw.stock.yahoo.com/news_content/url/d/...,跌
1,1451864508513_N01,news,yahoo股市,重大要聞,2016-01-04 07:24:00,經濟頭條~台積16奈米 完勝三星,中央社,半導體設備商透露，三星近期已通知設備相關協力廠，暫緩邏輯晶片廠擴建腳步，主因三星考量旗下14...,https://tw.stock.yahoo.com/news_content/url/d/...,跌
2,1451865407565_N01,news,yahoo股市,重大要聞,2016-01-04 07:34:00,《半導體》i7捎單有影，台積電營運逐季旺,時報資訊,蘋果新一代智慧型手機iPhone 7將在今年下半年登場，根據外電消息，蘋果自行設計的A10應...,https://tw.stock.yahoo.com/news_content/url/d/...,跌


In [8]:
tnews['label'].fillna(value='哈哈', inplace=True)
tnews1=tnews[tnews['label'].str.contains('漲')]
tnews2=tnews[tnews['label'].str.contains('跌')]
tnews3=tnews[tnews['label'].str.contains('平')]
tnews4=tnews[tnews['label'].str.contains('哈哈')]
print(len(tnews),len(tnews1),len(tnews2),len(tnews3),len(tnews4))

3430 1716 1414 272 28


In [9]:
tnews = tnews[tnews['label'].str.contains('漲|跌')]
tnews = tnews.reset_index(drop=True)
len(tnews)

3130

In [10]:
df_news = tnews[['title','content','label']]
df_news.head()

,title,content,label
0,台積電ADR31日下跌0.13美元跌幅0.57%折台股150.45元,(中央社台北2016年 1月 1日電)台積電在NYSE掛牌 ADR以 22.75美元作收，下...,跌
1,經濟頭條~台積16奈米 完勝三星,半導體設備商透露，三星近期已通知設備相關協力廠，暫緩邏輯晶片廠擴建腳步，主因三星考量旗下14...,跌
2,《半導體》i7捎單有影，台積電營運逐季旺,蘋果新一代智慧型手機iPhone 7將在今年下半年登場，根據外電消息，蘋果自行設計的A10應...,跌
3,經濟日報頭條：台積16奈米，完勝三星,半導體設備商透露，三星近期已通知設備相關協力廠，暫緩邏輯晶片廠擴建腳步，主因三星考量旗下14...,跌
4,工商證券頭條：i7捎單有影，台積電營運逐季旺,蘋果新一代智慧型手機iPhone 7將在2016年下半年登場，根據外電消息，蘋果自行設計的A...,跌


In [11]:
# 0是跌，1是漲
df_news.loc[df_news['label']=='漲', 'num_label'] = 1
df_news.loc[df_news['label']=='跌', 'num_label'] = 0
df_news.head()

/home/daniel/anaconda3/envs/bertenv/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/daniel/anaconda3/envs/bertenv/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,title,content,label,num_label
0,台積電ADR31日下跌0.13美元跌幅0.57%折台股150.45元,(中央社台北2016年 1月 1日電)台積電在NYSE掛牌 ADR以 22.75美元作收，下...,跌,0.0
1,經濟頭條~台積16奈米 完勝三星,半導體設備商透露，三星近期已通知設備相關協力廠，暫緩邏輯晶片廠擴建腳步，主因三星考量旗下14...,跌,0.0
2,《半導體》i7捎單有影，台積電營運逐季旺,蘋果新一代智慧型手機iPhone 7將在今年下半年登場，根據外電消息，蘋果自行設計的A10應...,跌,0.0
3,經濟日報頭條：台積16奈米，完勝三星,半導體設備商透露，三星近期已通知設備相關協力廠，暫緩邏輯晶片廠擴建腳步，主因三星考量旗下14...,跌,0.0
4,工商證券頭條：i7捎單有影，台積電營運逐季旺,蘋果新一代智慧型手機iPhone 7將在2016年下半年登場，根據外電消息，蘋果自行設計的A...,跌,0.0


In [12]:
df_news['num_label'] = df_news['num_label'].astype(int)
df_news['title'] = df_news['title'].astype(str)
df_news['content'] = df_news['content'].astype(str)
df_news.head()

/home/daniel/anaconda3/envs/bertenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/daniel/anaconda3/envs/bertenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/daniel/anaconda3/envs/bertenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

,title,content,label,num_label
0,台積電ADR31日下跌0.13美元跌幅0.57%折台股150.45元,(中央社台北2016年 1月 1日電)台積電在NYSE掛牌 ADR以 22.75美元作收，下...,跌,0
1,經濟頭條~台積16奈米 完勝三星,半導體設備商透露，三星近期已通知設備相關協力廠，暫緩邏輯晶片廠擴建腳步，主因三星考量旗下14...,跌,0
2,《半導體》i7捎單有影，台積電營運逐季旺,蘋果新一代智慧型手機iPhone 7將在今年下半年登場，根據外電消息，蘋果自行設計的A10應...,跌,0
3,經濟日報頭條：台積16奈米，完勝三星,半導體設備商透露，三星近期已通知設備相關協力廠，暫緩邏輯晶片廠擴建腳步，主因三星考量旗下14...,跌,0
4,工商證券頭條：i7捎單有影，台積電營運逐季旺,蘋果新一代智慧型手機iPhone 7將在2016年下半年登場，根據外電消息，蘋果自行設計的A...,跌,0


In [13]:
# Load pre-trained model tokenizer, to convert our text into tokens that correspond to BERT’s vocabulary.
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

## 下面是小小的測試bert (tokens tensor+segments_tensor)

In [14]:
# Tokenize input
text = "很好看的動作片，不會浪費錢跟時間。很久沒有這樣的探險片。可說是女版的印第安那瓊。"
tokens = tokenizer.tokenize(text)      # 每個字切詞成一個list
print(type(tokens))                 # list
np.array(tokens)                    # 轉成numpy

<class 'list'>


array(['很', '好', '看', '的', '動', '作', '片', '，', '不', '會', '浪', '費', '錢',
       '跟', '時', '間', '。', '很', '久', '沒', '有', '這', '樣', '的', '探', '險',
       '片', '。', '可', '說', '是', '女', '版', '的', '印', '第', '安', '那', '瓊',
       '。'], dtype='<U1')

In [15]:
input_ids = tokenizer.convert_tokens_to_ids(tokens)   # 每個字轉成id
print(type(input_ids))                         # list
print(len(input_ids))
np.array(input_ids)

<class 'list'>
40


array([2523, 1962, 4692, 4638, 1240,  868, 4275, 8024,  679, 3298, 3857,
       6527, 7092, 6656, 3229, 7279,  511, 2523,  719, 3760, 3300, 6857,
       3564, 4638, 2968, 7402, 4275,  511, 1377, 6303, 3221, 1957, 4276,
       4638, 1313, 5018, 2128, 6929, 4475,  511])

In [16]:
token_type_ids = tokenizer.create_token_type_ids_from_sequences(input_ids) # token_type_ids 必須input還沒加CLS SEP
print(type(token_type_ids))                                # list
print(len(token_type_ids)) 
np.array(token_type_ids)
# 多兩個token是CLS、SEP

<class 'list'>
42


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [17]:
input_ids = tokenizer.build_inputs_with_special_tokens(input_ids)    # 句子前後加上 CLS SEP 的 id
print(type(input_ids))
print(len(input_ids))
np.array(input_ids)
# 101是CLS，101是SEP

<class 'list'>
42


array([ 101, 2523, 1962, 4692, 4638, 1240,  868, 4275, 8024,  679, 3298,
       3857, 6527, 7092, 6656, 3229, 7279,  511, 2523,  719, 3760, 3300,
       6857, 3564, 4638, 2968, 7402, 4275,  511, 1377, 6303, 3221, 1957,
       4276, 4638, 1313, 5018, 2128, 6929, 4475,  511,  102])

In [18]:
n = 512 - len(input_ids)
input_ids2 = np.pad(input_ids, (0, n), mode ='constant', constant_values=(0))  # array右邊append n 個 0  補長度到512
print(len(input_ids2))
input_ids2

512


array([ 101, 2523, 1962, 4692, 4638, 1240,  868, 4275, 8024,  679, 3298,
       3857, 6527, 7092, 6656, 3229, 7279,  511, 2523,  719, 3760, 3300,
       6857, 3564, 4638, 2968, 7402, 4275,  511, 1377, 6303, 3221, 1957,
       4276, 4638, 1313, 5018, 2128, 6929, 4475,  511,  102,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

## 把上述處理建立一個function，丟入一串文字試試

In [19]:
# 把維度固定在512維
def input_ids_all(text):
    if len(text)>510:
        text = text[0:510]        
    tokens = tokenizer.tokenize(text)                          # 每個字切詞成一個list
    input_ids = tokenizer.convert_tokens_to_ids(tokens)        # 每個字轉成id
    input_ids = tokenizer.build_inputs_with_special_tokens(input_ids)    # 句子前後加上 CLS SEP 的 id
    input_ids = np.array(input_ids)                            # list 轉 numpy
    if len(input_ids) < 512:
        n = 512 - len(input_ids)
        input_ids = np.pad(input_ids, (0, n), mode ='constant', constant_values=(0))    # array右邊append n 個 0  補長度到512
    return input_ids

In [25]:
# 把維度固定再64維
def input_ids_all_64(text):
    if len(text)>62:
        text = text[0:62]        
    tokens = tokenizer.tokenize(text)                          # 每個字切詞成一個list
    input_ids = tokenizer.convert_tokens_to_ids(tokens)        # 每個字轉成id
    input_ids = tokenizer.build_inputs_with_special_tokens(input_ids)    # 句子前後加上 CLS SEP 的 id
    input_ids = np.array(input_ids)                            # list 轉 numpy
    if len(input_ids) < 64:
        n = 64 - len(input_ids)
        input_ids = np.pad(input_ids, (0, n), mode ='constant', constant_values=(0))    # array右邊append n 個 0  補長度到512
    return input_ids

In [33]:
text = "很好看的動作片，不會浪費錢跟時間。很久沒有這樣的探險片。可說是女版的印第安那瓊。"
input_ids_all_64(text)

array([ 101, 2523, 1962, 4692, 4638, 1240,  868, 4275, 8024,  679, 3298,
       3857, 6527, 7092, 6656, 3229, 7279,  511, 2523,  719, 3760, 3300,
       6857, 3564, 4638, 2968, 7402, 4275,  511, 1377, 6303, 3221, 1957,
       4276, 4638, 1313, 5018, 2128, 6929, 4475,  511,  102,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

## attention mask tensors

In [22]:
def attention_mask_all(text):
    tokens = tokenizer.tokenize(text)       # 每個字切詞成一個list
    input_ids = tokenizer.convert_tokens_to_ids(tokens)  # 每個字轉成id
    input_ids = tokenizer.build_inputs_with_special_tokens(input_ids)    # 句子前後加上 CLS SEP 的 id
    input_ids = np.array(input_ids)          # list 轉 numpy
    attention_mask = np.array([1,1])
    attention_mask = np.pad(attention_mask, (0, len(input_ids)-2), mode ='constant', constant_values=(1)) 
#     array右邊append 1 到跟segment一樣長
    if len(attention_mask) < 512:
        n = 512 - len(attention_mask)
        attention_mask = np.pad(attention_mask, (0, n), mode ='constant', constant_values=(0))  # array右邊append n 個 0  補長度到512
    return attention_mask

In [23]:
attention_mask_all(text)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## 接下來要開始動真格了

In [34]:
text = df_news['title'].to_numpy()
print(len(text))
text

3130


array(['台積電ADR31日下跌0.13美元跌幅0.57%折台股150.45元', '經濟頭條~台積16奈米 完勝三星',
       '《半導體》i7捎單有影，台積電營運逐季旺', ...,
       '產業：企業有意在2020年前部署5G，台積、京元電、台燿、聯茂、先豐最先受益', '【Y晚報】台積電領軍 指數收復9600點',
       '《證交所》今日單一證券鉅額成交股票：台苯、台積電'], dtype=object)

In [35]:
type(text)

numpy.ndarray

In [36]:
input_ids = [input_ids_all_64(i) for i in text]      # 必須要 [ ] 輸出是list
input_ids = np.array(input_ids)                    # 轉成numpy
input_ids

array([[ 101, 1378, 4948, ...,    0,    0,    0],
       [ 101, 5195, 4089, ...,    0,    0,    0],
       [ 101,  517, 1288, ...,    0,    0,    0],
       ...,
       [ 101, 4496, 3511, ...,    0,    0,    0],
       [ 101,  523,  100, ...,    0,    0,    0],
       [ 101,  517, 6349, ...,    0,    0,    0]])

In [41]:
input_ids[0]

array([ 101, 1378, 4948, 7442,  100, 3189,  678, 6649,  121,  119, 8124,
       5401, 1039, 6649, 2388,  121,  119, 8272,  110, 2835, 1378, 5500,
       8269,  119, 8208, 1039,  102,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

In [42]:
label = df_news['num_label'].to_numpy()
label

array([0, 0, 0, ..., 1, 1, 1])

In [43]:
from sklearn.model_selection import train_test_split
# Use train_test_split to split our data into train and validation sets for training

# # 設定 stratify = label 把每個類別平均
train_input_ids, validation_input_ids, train_label, validation_label = train_test_split(input_ids, label, 
                                                            random_state=2018, test_size=0.2, stratify=label )

In [44]:
# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule 
# num_labels=5 分5類
model = TFBertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=2)
model.summary()


optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  102267648 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 102,269,186
Trainable params: 102,269,186
Non-trainable params: 0
_________________________________________________________________


In [45]:
# Train and evaluate using tf.keras.Model.fit()  # batch size 8就會error 可能是記憶體爆掉
model_fit = model.fit(train_input_ids, train_label, 
                      batch_size=4, epochs=4, 
                    validation_data=(validation_input_ids, validation_label)
#                    steps_per_epoch=115,
#                    validation_steps=7)
                   )

Train on 2504 samples, validate on 626 samples
Epoch 1/4
2504/2504 [==============================] - 36s 14ms/sample - loss: 0.7043 - accuracy: 0.5140 - val_loss: 0.6889 - val_accuracy: 0.5479
Epoch 2/4
2504/2504 [==============================] - 26s 10ms/sample - loss: 0.6989 - accuracy: 0.5383 - val_loss: 0.6883 - val_accuracy: 0.5479
Epoch 3/4
2504/2504 [==============================] - 26s 10ms/sample - loss: 0.7017 - accuracy: 0.5248 - val_loss: 0.6900 - val_accuracy: 0.5479
Epoch 4/4
2504/2504 [==============================] - 26s 10ms/sample - loss: 0.6995 - accuracy: 0.5236 - val_loss: 0.6886 - val_accuracy: 0.5479
